# CSC8002 Assignment 2

## subset_nosql_database_creation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import pymongo
from pymongo import MongoClient


import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


import datetime

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
# Load state measures data
measuresdf = pd.read_csv('C:\\Users\Justin\Desktop\Semester 3 2020\CSC8002 Big Data Management\Assignments\measures.csv')

# Load created subset of Google Mobility Data
coviddfausnatisoclean = pd.read_csv('C:\\Users\Justin\Desktop\Semester 3 2020\CSC8002 Big Data Management\Assignments\coviddfausnatisoclean.csv')

In [6]:
measuresdf

,iso_3166_2_code,date,measure
0,AU-ACT,2020-03-19,visitor restrictions/events cancelled
1,AU-ACT,2020-03-23,Non-essential acitivities closed
2,AU-ACT,2020-03-25,elective surgeries suspended
3,AU-ACT,2020-04-08,public schools remote learning
4,AU-ACT,2020-04-24,elective surgeries resume
5,AU-ACT,2020-05-01,Non-essential acitivities partial restriction
6,AU-ACT,2020-05-12,Non-essential acitivities further restored
7,AU-ACT,2020-05-26,Non-essential acitivities further restored
8,AU-NSW,2020-03-15,visitor restrictions/events cancelled
9,AU-NSW,2020-03-18,Non-essential acitivities partial restriction


In [5]:
coviddfausnatisoclean

,Unnamed: 0,iso_3166_2_code,date,Mean_Essential,Mean_Non_Essential
0,150462,AU-ACT,2020-02-15,2.0,11.25
1,150463,AU-ACT,2020-02-16,8.5,15.00
2,150464,AU-ACT,2020-02-17,3.0,13.50
3,150465,AU-ACT,2020-02-18,-1.5,5.75
4,150466,AU-ACT,2020-02-19,3.0,13.25
...,...,...,...,...,...
2323,215111,AU-WA,2020-11-27,3.5,-8.25
2324,215112,AU-WA,2020-11-28,3.5,-7.50
2325,215113,AU-WA,2020-11-29,3.5,-14.25
2326,215114,AU-WA,2020-11-30,5.0,-16.00


### Upload

In [7]:
client = MongoClient('ec2-54-173-219-43.compute-1.amazonaws.com',
 443,
 username = "csc8002user",
 password = "csc8002password")

In [8]:
client.server_info()

{'allocator': 'tcmalloc',
 'bits': 64,
 'buildEnvironment': {'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.2.0',
  'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -fasynchronous-unwind-tables -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp',
  'cppdefines': 'SAFEINT_USE_INTRINSICS 0 PCRE_STATIC NDEBUG _XOPEN_SOURCE 700 _GNU_SOURCE _FORTIFY_SOURCE 2 BOOST_THREAD_VERSION 5 BOOST_THREAD_USES_DATETIME BOOST_SYSTEM_NO_DEPRECATED BOOST_MATH_NO_LONG_DOUBLE_MATH_FUNCTIONS BOOST_ENABLE_ASSERT_DEBUG_HANDLER BOOST_LOG_NO_SHORTHAND_NAMES BOOST_LOG_USE_NATIVE_SYSLOG BOOST_LOG_WITHOUT_THREAD_ATTR ABSL_FORCE_ALIGNED_ACCESS',
  'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.2.0',
  'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -fsized-de

In [9]:
client.list_database_names()

['admin', 'assignment2', 'config', 'local', 'test1']

In [150]:
db = client.assignment2
db.State_Measures.insert_many(measuresdf.to_dict('records'))
db.State_Mobility.insert_many(coviddfausnatisoclean.to_dict('records'))